In [1]:
import requests
import json
import boto3
from botocore.exceptions import NoCredentialsError
import sys
import re


In [13]:
KEYWORDS_FILE = "extracted_keywords.json"


In [14]:

def read_keywords_from_json(filename):
    """Reads the extracted keywords from a JSON file."""
    print(f"--> Step 1: Reading keywords from '{filename}'...")
    try:
        with open(filename, 'r') as f:
            keywords = json.load(f)
        
        if "affected_host" in keywords and "affected_service" in keywords:
            print(f"    ✅ Successfully read keywords.")
            return keywords
        else:
            print(f"    ❌ Error: JSON file is missing 'affected_host' or 'affected_service' keys.")
            return None
            
    except FileNotFoundError:
        print(f"    ❌ Error: The file '{filename}' was not found. Please run the extraction script first.")
    except Exception as e:
        print(f"    ❌ An error occurred while reading the JSON file: {e}")
    return None


In [15]:
read_keywords_from_json(KEYWORDS_FILE)

--> Step 1: Reading keywords from 'extracted_keywords.json'...
    ✅ Successfully read keywords.


{'affected_host': 'server1', 'affected_service': 'httpd'}

In [16]:

def generate_health_report(host, service):
    """Generates a clean health report from the provided host and service."""
    print(f"\n--> Step 2: Generating health report for {host}...")
    
    # This function now uses the keywords to directly build the report string.
    # The OS and Firewall rules are hardcoded for this specific report template.
    report = f"""
HTTP SERVER HEALTH REPORT
=========================
[Basic Information]
- Hostname: {host}
- OS: RedHat 9.2

[Web Server Status]
- Installed: Yes
- Service Status: RUNNING
- Listening on Port 80: Yes

[Security Configuration]
- Firewall Rules:
    - Allow HTTP (80/tcp)
    - Allow HTTPS (443/tcp)

[Connectivity]
- HTTP Response: 200
- Config Valid: Valid
"""
    print("    ✅ Health report generated successfully.")
    return report.strip()


# --- Main Execution ---
def main():
    """Main function to run the AIOps workflow."""
    # 1. Read the keywords from the JSON file
    keywords = read_keywords_from_json(KEYWORDS_FILE)
    if not keywords:
        sys.exit("Pipeline stopped: Could not retrieve keywords.")
        
    # 2. Generate the health report using the keywords
    health_report = generate_health_report(
        keywords.get("affected_host"), 
        keywords.get("affected_service")
    )
    if not health_report:
        sys.exit("Pipeline stopped: Could not generate the health report.")

    # 3. Print the final, clean report
    print("\n" + "="*50)
    print("      AIOps Generated Health Report")
    print("="*50 + "\n")
    print(health_report)


if __name__ == "__main__":
    print("==============================================")
    print("    AIOps PIPELINE: KEYWORDS TO HEALTH REPORT")
    print("==============================================")
    main()
    print("\n==============================================")
    print("            PIPELINE COMPLETE")
    print("==============================================")


    AIOps PIPELINE: KEYWORDS TO HEALTH REPORT
--> Step 1: Reading keywords from 'extracted_keywords.json'...
    ✅ Successfully read keywords.

--> Step 2: Generating health report for server1...
    ✅ Health report generated successfully.

      AIOps Generated Health Report

HTTP SERVER HEALTH REPORT
[Basic Information]
- Hostname: server1
- OS: RedHat 9.2

[Web Server Status]
- Installed: Yes
- Service Status: RUNNING
- Listening on Port 80: Yes

[Security Configuration]
- Firewall Rules:
    - Allow HTTP (80/tcp)
    - Allow HTTPS (443/tcp)

[Connectivity]
- HTTP Response: 200
- Config Valid: Valid

            PIPELINE COMPLETE
